In [1]:
#pip install python-dotenv

In [2]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [3]:
import os
from dotenv import load_dotenv

In [4]:
import pandas as pd
import json
import csv

In [5]:
load_dotenv()

True

In [6]:
sp_clave = os.getenv("miriam_id_spotify")
sp_contraseña = os.getenv("miriam_secret_spotify")

In [7]:
#Autentificación:
auth_manager = sp = SpotifyClientCredentials(client_id=sp_clave,
                                             client_secret=sp_contraseña)

sp = spotipy.Spotify(auth_manager=auth_manager)

In [9]:
datos_pop = sp.search(q="genre:pop, year:2019", type="track", limit=50, offset=0)

In [10]:
canciones_pop = {"nombre_artista": [], "nombre_cancion": [], "año_lanzamiento": [], 'género_musical': [], 'tipo': []}
for anio in range(2019,2024):
    for offset in range(0, 250, 50):
        datos_pop = sp.search(q=f"genre:pop, year:{anio}", type="track", limit=50, offset=offset)
        for cancion in datos_pop["tracks"]["items"]:
            canciones_pop["nombre_artista"].append(cancion["album"]["artists"][0]["name"])
            canciones_pop["nombre_cancion"].append(cancion["name"])
            canciones_pop["año_lanzamiento"].append(cancion["album"]["release_date"][0:4])
            canciones_pop["género_musical"].append('pop')
            canciones_pop["tipo"].append(cancion["type"])
df_pop = pd.DataFrame(canciones_pop)
df_pop

,nombre_artista,nombre_cancion,año_lanzamiento,género_musical,tipo
0,Various Artists,Un boomerang,2019,pop,track
1,Los Rebeldes,Land of Thousand Dances - Remasterizado,2019,pop,track
2,Various Artists,No te enamores,2019,pop,track
3,Various Artists,Walk Me Home,2019,pop,track
4,Various Artists,Wow.,2019,pop,track
...,...,...,...,...,...
1245,Ajru Singer Mewati,Ajru Singer SR 4949,2023,pop,track
1246,Various Artists,Tà Áo Xanh - Dang Dở,2023,pop,track
1247,Covil,Os Aliado,2023,pop,track
1248,Nofin Asia,DJ ASOMASOW KHARIS SOPAN,2023,pop,track


In [15]:
df_pop.to_csv('pop_spotifydatabase.csv')

In [16]:
df_values=pd.read_csv("pop_spotifydatabase.csv",index_col=0)

In [17]:
list(df_values.itertuples(index=False, name=None))

[('Various Artists', 'Un boomerang', 2019, 'pop', 'track'),
 ('Los Rebeldes',
  'Land of Thousand Dances - Remasterizado',
  2019,
  'pop',
  'track'),
 ('Various Artists', 'No te enamores', 2019, 'pop', 'track'),
 ('Various Artists', 'Walk Me Home', 2019, 'pop', 'track'),
 ('Various Artists', 'Wow.', 2019, 'pop', 'track'),
 ('Los Rebeldes',
  'Mediterraneo (Ensayos Básicamente) - Remasterizado',
  2019,
  'pop',
  'track'),
 ('Various Artists', 'Candela - Latin Workout Remix', 2019, 'pop', 'track'),
 ('VINAI', 'Break the Beat - Radio Edit', 2019, 'pop', 'track'),
 ('Various Artists', 'Déjame Ser', 2019, 'pop', 'track'),
 ('Chaqueta de Chándal',
  'A Moderno Resabiado No Le Mires el Dentado',
  2019,
  'pop',
  'track'),
 ('Various Artists', 'Bonita', 2019, 'pop', 'track'),
 ('Marisol', 'Corre, Corre Caballito - Remasterizado', 2019, 'pop', 'track'),
 ('Various Artists', 'Malibu', 2019, 'pop', 'track'),
 ('Da Igual', 'Eres Tú', 2019, 'pop', 'track'),
 ('Nada Surf', 'Something I Should 

In [18]:
values = list(df_values.itertuples(index=False, name=None))

In [19]:
import mysql.connector
from mysql.connector import errorcode

In [20]:
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', database='MusicTrends')


mycursor = cnx.cursor()
sql = "INSERT INTO Spotify (nombre_artista, nombre_cancion, año_lanzamiento, género_musical, tipo) VALUES (%s, %s, %s, %s,%s)"
val = values

try: 
    mycursor.executemany(sql, val)
    cnx.commit()
    print(mycursor.rowcount, "registro/s insertado/s.")

except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

1292 (22007): Incorrect date value: '2019' for column 'año_lanzamiento' at row 1
Error Code: 1292
SQLSTATE 22007
Message Incorrect date value: '2019' for column 'año_lanzamiento' at row 1
